# Trie（字典树、前缀树）
- 一种多叉树
- 通常只用来处理字符串，查询复杂度只和字符串的长度城正比，为O(w)，w为查询单词的长度
- 本节代码实现两种版本的字典树——递归版本和非递归版本，并增加了删除与打印操作，有不足之处还请指出，共同学习哈
- python dict的底层实现是红黑树，查找操作的时间复杂度为O(1)
- 本节实现的前缀树的成员方法我都没对输入进行是否是字符串的检查，请大家在传值的时候传字符串就好。

In [1]:
# Time 2019-04-06
class Node:
    def __init__(self, isword=False):
        """
        节点类的构造函数
        Params:
            - isword: bool值，当前字符位置是否是一个有效字符串的标识，比如pan和panda。默认值为False
            - next_: 一般是一个python字典，键为一个字符，即边，值为边所指向的Node。默认值为空字典
        """
        self.isword = isword
        self.next = dict()

In [2]:
class Trie:
    def __init__(self):
        """
        Trie类的构造函数
        两个成员变量root和size
        """
        self.root = Node()
        self.size = 0
        
    def getSize(self):
        """查看size"""
        return self.size
    
    def add_nr(self, astring):
        """
        添加新字符串的非递归写法
        O(w)
        Params:
            - astring: 待添加的字符串
        """
        cur_node = self.root
        for elem in astring:
            if cur_node.next.get(elem, None) is None:
                cur_node.next[elem] = Node()
            cur_node = cur_node.next[elem]
        if not cur_node.isword:
            cur_node.isword = True
            self.size += 1
            
    def add(self, astring):
        """
        添加新字符串的递归写法
        O(w)
        """
        self._add(self.root, astring, 0)
        
    def contains_nr(self, astring):
        """
        判断一个字符串是否已经存在于Trie中的非递归写法
        O(w)
        Params:
            - astring: 待查询的字符串
        Returns:
            存在为True，否则为False
        """
        cur_node = self.root
        for elem in astring:
            if cur_node.next.get(elem, None) is None:
                return False
            cur_node = cur_node.next[elem]
        return cur_node.isword
        
    def contains(self, string):
        """
        判断一个字符串是否已经存在于Trie中的递归写法
        O(w)
        """
        return self._contains(self.root, string, 0)
    
    def isPrefix_nr(self, astring):
        """
        判断某个字符串是否是当前Trie的一个前缀，非递归写法
        O(w)
        Params:
            - astring: 输入的字符串
        Returns:
            是前缀的话返回True，否则返回False
        """
        cur_node = self.root
        for elem in astring:
            if cur_node.next.get(elem, None) is None:
                return False
            cur_node = cur_node.next[elem]
        return True  # 注意前缀就不用考虑isword了，只要存在这个字符串就行。
    
    def isPrefix(self, astring):
        """
        判断某个字符串是否是当前Trie的一个前缀，递归写法
        O(w)
        """
        return self._isPrefix(self.root, astring, 0)
    
    def remove_nr(self, astring):
        """
        删除Trie树中的一个字符串，一般用的很少，不掌握也没关系的
        remove方法这里我只写了非递归写法，递归写法和这个差不多，也需要往record_nodes里塞node，
        删除的时候还是要遍历的，差不多，只是前面的代码不一样，就不写了
        O(w)
        Params:
            - asting: 待删除的string
        """
        if not self.contains_nr(astring):
            return  # 不存在就什么都不做
        cur_node = self.root
        record_nodes = [(cur_node, cur_node.isword)] # 必须要存储根节点，因为首字母的边只有root携带
        for elem in astring:
            record_nodes.append((cur_node.next[elem], cur_node.next[elem].isword))
            cur_node = cur_node.next[elem]
            
        # 注意有可能删除的是一个前缀！此时的处理方法很简单，isword变成False就完事了
        if len(cur_node.next):
            cur_node.isword = False
            self.size -= 1
            return
        
        # 反着删就完事了
        # 注意如果同时存在'pan'和'pandas'，删除'pandas'后'pan'不应该受到影响，所以存储isword的作用就在这里
        # 此时reocrd_nodes的长度比astring的长度多1，因为最后一个元素会是一个空的Node，它的isword一定为True
        # 找到记录的信息中最后一个isword为True的Node，然后删除它后面的就可以了
        index = len(astring) - 1
        while index >= 0:
            if record_nodes[index][1] == True:
                # 此时到达终止条件，此时还需要继续删1次，就完全删干净了，想一想为什么要再删一次，好好debug一下,删完退出循环
                del record_nodes[index][0].next[astring[index]] 
                break
            else:
                del record_nodes[index][0].next[astring[index]] # 从后往前依次删
            index -= 1
        self.size -= 1  # 最后维护一下self.size就完事了
    
    def print_(self):
        """
        打印字典树中的全部元素
        '卍' 代表字符串彻底完结
        'θ' 代表某个字符串的isword为True，此时有可能是一个前缀
        其实这里的打印仅仅是debug，我也实在想不出来特别好的办法来打印前缀了，上面两个字符在应用场景
        中还是可能会用到的，此时就会造成打印错误，这里仅仅是debug看一下就好
        其实打印的话还是学到它递归的思想就好了，我这个打印方法可能很low/(ㄒoㄒ)/~~
        """
        def _printTrie(node, alist):
            """
            打印以node为根节点的Trie树
            Params:
                - node: 输入的根节点
            """
            if not len(node.next):
                alist.append('卍')
                return
            if node.isword:
                alist.append('θ')
            for key in node.next.keys():
                alist.append(key)
                _printTrie(node.next[key], alist)
                
        record_list = []
        _printTrie(self.root, record_list)
        total_strings = ''.join(record_list)
        split_strings = total_strings.split('卍')[:-1] # 因为最后一个字符必为'卍'，我们就不要了
        print('[', end=' ')
        for split_index, split_string in enumerate(split_strings):
            pre_string = split_string.split('θ')
            temp_elem = ''
            for elem_index, elem in enumerate(pre_string):
                temp_elem += elem
                if (split_index == len(split_strings) - 1) and (elem_index == len(pre_string) - 1):
                    print(temp_elem, end=' ')
                else:
                    print(temp_elem, end=',')
        print(']')
        
        
    # private 
    def _add(self, node, astring, index):
        """
        添加新字符串到以node为根的Trie中的递归写法
        O(w)
        Params:
            - node: 当前根节点
            - astring: 待添加的字符串
            - index: 即将添加的astring中字符的索引
        """
        if index == len(astring) and not node.isword:
            node.isword = True
            self.size += 1
            return

        if node.next.get(astring[index], None) is None:
            node.next[astring[index]] = Node()
        return self._add(node.next[astring[index]], astring, index + 1)

    def _contains(self, node, astring, index):
        """
        查询字符串是否存在于以node为根的Trie中，递归写法
        O(w)
        Params:
            - node: 当前根节点
            - astring: 待查询的字符串
            - index: 即将查询的astring中字符的索引
        Returns:
            存在返回True，否则返回False
        """
        if index == len(astring) and node.isword is True:
            return True
        if node.next.get(astring[index], None) is None:
            return False
        return self._contains(node.next[astring[index]], astring, index + 1)
    
    def _isPrefix(self, node, astring, index):
        """
        判断某个字符串是否是当前Trie的一个前缀，递归写法
        Params:
            - node: 当前根节点
            - astring: 待考察的字符串
            - index: 即将考察的astring中字符的索引
        """
        if index == len(astring):
            return True
        if node.next.get(astring[index], None) is None:
            return False
        return self._isPrefix(node.next[astring[index]], astring, index + 1)

### 测试非递归版本Trie树的成员函数

In [5]:
# test trie_nr
# python的强大之处，只要是字符就行O(∩_∩)O，真的不懂什么叫ASCII码啊(｀・ω・´)
record_strings = ['xi', 'huan', 'bobo', 'laoshi', 'pan', 'pandas', 'やめて']
test_trie_nr = Trie()
print('将record_strings中的元素添加进Trie中-----', end=' ')
for elem in record_strings:
    test_trie_nr.add_nr(elem)
test_trie_nr.print_()
print('此时所有的字符串是否已经全部添加进test_trie_nr中？-----', end=' ')
print('{', end=' ')
for elem in record_strings:
    flag = '存在' if test_trie_nr.contains_nr(elem) else '不存在'
    print('"{}"{}于test_trie_nr中'.format(elem, flag), end=', ')
print('}')
print('此时的size-----', test_trie_nr.getSize())
print('是否存在前缀"pand?"-----', test_trie_nr.isPrefix_nr('pand'))
print('从test_trie_nr中删除"pandas"-----', end=' ')
test_trie_nr.remove_nr('pandas')
test_trie_nr.print_()
print('test_trie_nr中是否包含"pandas"?-----', test_trie_nr.contains_nr('pandas'))
print('此时的size-----', test_trie_nr.getSize())

将record_strings中的元素添加进Trie中----- [ xi,huan,bobo,laoshi,pan,pandas,やめて ]
此时所有的字符串是否已经全部添加进test_trie_nr中？----- { "xi"存在于test_trie_nr中, "huan"存在于test_trie_nr中, "bobo"存在于test_trie_nr中, "laoshi"存在于test_trie_nr中, "pan"存在于test_trie_nr中, "pandas"存在于test_trie_nr中, "やめて"存在于test_trie_nr中, }
此时的size----- 7
是否存在前缀"pand?"----- True
从test_trie_nr中删除"pandas"----- [ xi,huan,bobo,laoshi,pan,やめて ]
test_trie_nr中是否包含"pandas"?----- False
此时的size----- 6


### 测试递归版本Trie树的成员函数

In [4]:
# test trie
record_strings = ['xi', 'huan', 'bobo', 'laoshi', 'pan', 'pandas']
test_trie = Trie()
print('将record_strings中的元素添加进Trie中-----', end=' ')
for elem in record_strings:
    test_trie.add(elem)
test_trie.print_()
print('此时所有的字符串是否已经全部添加进test_trie_nr中？-----', end=' ')
print('{', end=' ')
for elem in record_strings:
    flag = '存在' if test_trie.contains(elem) else '不存在'
    print('"{}"{}于test_trie_nr中'.format(elem, flag), end=', ')
print('}')
print('此时的size-----', test_trie.getSize())
print('是否存在前缀"pand?"-----', test_trie.isPrefix('pand'))

将record_strings中的元素添加进Trie中----- [ xi,huan,bobo,laoshi,pan,pandas ]
此时所有的字符串是否已经全部添加进test_trie_nr中？----- { "xi"存在于test_trie_nr中, "huan"存在于test_trie_nr中, "bobo"存在于test_trie_nr中, "laoshi"存在于test_trie_nr中, "pan"存在于test_trie_nr中, "pandas"存在于test_trie_nr中, }
此时的size----- 6
是否存在前缀"pand?"----- True
